In [25]:
import numpy as np
np.log(3)

1.0986122886681098

In [20]:
%%writefile kltext_1.txt
1.Data Science is an interdisciplinary field about processes and systems to extract knowledge or insights from large volumes of data in various forms (data in various forms, data in various forms, data in various forms), either structured or unstructured,[1][2] which is a continuation of some of the data analysis fields such as statistics, data mining and predictive analytics, as well as Knowledge Discovery in Databases.
2.Machine learning is a subfield of computer science[1] that evolved from the study of pattern recognition and computational learning theory in artificial intelligence.[1] Machine learning explores the study and construction of algorithms that can learn from and make predictions on data.[2] Such algorithms operate by building a model from example inputs in order to make data-driven predictions or decisions,[3]:2 rather than following strictly static program instructions.

Writing kltext_1.txt


In [21]:
%%writefile kltext_2.txt
1.Global warming is already having significant and harmful effects on our communities, our health, and our climate. Sea level rise is accelerating. The number of large wildfires is growing. Dangerous heat waves are becoming more common. Extreme storm events are increasing in many areas. More severe droughts are occurring in others. We must take immediate action to address global warming or these consequences will continue to intensify, grow ever more costly, and increasingly affect the entire planet—including you, your community, and your family.
2. Global warming, also referred to as climate change, is the observed century-scale rise in the average temperature of the Earth's climate system and its related effects.[1][2] Multiple lines of scientific evidence show that the climate system is warming.[3][4][5] Many of the observed changes since the 1950s are unprecedented in the instrumental temperature record which extends back to the mid-19th century, and in paleoclimate proxy records covering thousands of years.[6]

Writing kltext_2.txt


# TWO DATA SETS, USE ONE OR THE OTHER

In [16]:
!cat kltext_1.txt

1.Data Science is an interdisciplinary field about processes and systems to extract knowledge or insights from large volumes of data in various forms (data in various forms, data in various forms, data in various forms), either structured or unstructured,[1][2] which is a continuation of some of the data analysis fields such as statistics, data mining and predictive analytics, as well as Knowledge Discovery in Databases.
2.Machine learning is a subfield of computer science[1] that evolved from the study of pattern recognition and computational learning theory in artificial intelligence.[1] Machine learning explores the study and construction of algorithms that can learn from and make predictions on data.[2] Such algorithms operate by building a model from example inputs in order to make data-driven predictions or decisions,[3]:2 rather than following strictly static program instructions.

In [23]:
!cat kltext_2.txt

1.Global warming is already having significant and harmful effects on our communities, our health, and our climate. Sea level rise is accelerating. The number of large wildfires is growing. Dangerous heat waves are becoming more common. Extreme storm events are increasing in many areas. More severe droughts are occurring in others. We must take immediate action to address global warming or these consequences will continue to intensify, grow ever more costly, and increasingly affect the entire planet���including you, your community, and your family.
2. Global warming, also referred to as climate change, is the observed century-scale rise in the average temperature of the Earth's climate system and its related effects.[1][2] Multiple lines of scientific evidence show that the climate system is warming.[3][4][5] Many of the observed changes since the 1950s are unprecedented in the instrumental temperature record which extends back to the mid-19th century, and in paleoclimate proxy records

In [49]:
%%writefile kldivergence.py
# -*- coding: utf-8 -*-
from __future__ import division
from mrjob.job import MRJob
import re
import numpy as np
class kldivergence(MRJob):
    
    # process each string character by character
    # the relative frequency of each character emitting Pr(character|str)
    # for input record 1.abcbe
    # emit "a"    [1, 0.2]
    # emit "b"    [1, 0.4] etc...
    def mapper1(self, _, line):
        index = int(line.split('.',1)[0])
        letter_list = re.sub(r"[^A-Za-z]+", '', line).lower()
        count = {}
        for l in letter_list:
            count[l] = count.get(l, 0) + 1

        # UNCOMMENT ONE OR THE OTHER "YIELD" DEPENDING UPON UNSMOTTHED OR
        # SMOOTHED OUTPUT
        
        # Without smoothing
        for key in count:
            #yield key, [index, count[key]*1.0/len(letter_list)]
        # with smoothing
            yield key, [index, (count[key] + 1) * 1.0 / (len(letter_list) + 24)]


    # on a component i calculate (e.g., "b")
    # Kullback–Leibler divergence of Q from P is defined
    #  (P(i) log (P(i) / Q(i))
    #
    def reducer1(self, key, values):
        p = 0
        q = 0
        for stringID in values:
            if stringID[0] == 1: # FIRST STRING
                p = stringID[1]
            else:                # SECOND STRING
                q = stringID[1]
            
            # FOUND THROUGH DEBUGGING THAT q CAN AT TIMES BE ZERO
            # HAVE TO INVESTIGATE, WHEN CAN q be 0.
            if q != 0:
                yield '*', p*np.log(p/q)
    
    def mapper2(self, key, values):
        yield key, values
        
    def reducer2(self, key, values):
        kl_sum = 0
        for value in values:
            kl_sum = kl_sum + value
        yield "KLDivergence", kl_sum
            
    def steps(self):
        return [self.mr(mapper=self.mapper1,reducer=self.reducer1),
                self.mr(mapper= self.mapper2, reducer=self.reducer2)
               ]

if __name__ == '__main__':
    kldivergence.run()
    

Overwriting kldivergence.py


In [50]:
%reload_ext autoreload
%autoreload 2
from mrjob.job import MRJob
from kldivergence import kldivergence

#dont forget to save kltext.txt (see earlier cell)
mr_job = kldivergence(args=['kltext_2.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

(u'KLDivergence', 0.0536547493)
